In [1]:
import torch
import torchvision
# import
import os
import pickle
import numpy as np
import PIL.Image
import cv2 

import scipy.io as sio
from datetime import datetime
import sys


sys.path.append('../model')
from resnet3d import resnet, mean
from C3D import C3D
                
sys.path.append('../icnn_torch')
from utils_upd import normalise_img, clip_extreme_pixel, save_video, normalise_vid, vid_preprocess,vid_deprocess, get_cnn_features
from icnn_lbfgs_upd import reconstruct_video_upd




In [2]:
img_mean = np.array(mean.get_mean())
img_std = np.array(mean.get_std())
norm  = 255

In [3]:
input_size = (90, 112,112, 3)
initial_input = np.random.randint(0, 256, input_size).astype(np.float)

In [4]:
video = vid_preprocess(initial_input, img_mean, img_std, norm)

In [5]:
video.shape

(3, 90, 112, 112)

In [38]:
net

DataParallel(
  (module): ResNet(
    (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (ide

In [6]:
net = resnet.resnet50(num_classes=400, shortcut_type='B', sample_size=112, sample_duration=90)
net = torch.nn.DataParallel(net, device_ids=None)
param_file = os.path.join('../model','resnet3d', 'resnet-50-kinetics.pth')
param = torch.load(param_file, map_location='cpu')
net.load_state_dict(param['state_dict'])
net.eval()

img_mean = np.array(mean.get_mean())
img_std = np.array(mean.get_std())
norm  = 255

../model/resnet3d/resnet.py:159: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [91]:
net = C3D.C3D_conv()
param_file = os.path.join('../model','C3D', 'c3d.pickle')
net.load_state_dict(torch.load(param_file))
net.eval()


img_mean = np.float32([104, 117, 123])
img_std = np.float32([1,1,1])
norm = 1

In [11]:
cap = cv2.VideoCapture('v009_0050.mp4')

In [12]:
org_video = []
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        org_video.append(cv2.resize(frame, (112, 112)).astype(np.float32))

    else:
        cap.release()
        break
org_video = np.array(org_video)
# preprocessing (mean substruction)
org_vid = vid_preprocess(org_video, img_mean, img_std,norm=norm)

inputs = torch.Tensor(org_vid[np.newaxis])

In [7]:
layer_list = ['conv1', 'layer1[1].iden', 'layer4[2].iden']
#layer_list = ['conv1']
exec_str_list = ["net.module."+layer +".register_forward_hook(hook)" for layer in layer_list]

In [95]:
#C3D
layer_list = ['conv1', 'conv4b']
exec_str_list = ["net."+layer +".register_forward_hook(hook)" for layer in layer_list]

In [13]:
features = get_cnn_features(net, inputs, exec_str_list)

In [14]:
len(features)

3

In [15]:
loss_fun = torch.nn.MSELoss()

In [16]:
input_size = (90, 112,112, 3)
initial_input = np.random.randint(0, 256, input_size).astype(np.float)

In [17]:
# input = 
video = vid_preprocess(initial_input, img_mean, img_std, norm)
inputs = torch.Tensor(video[np.newaxis])
inputs.requires_grad_()

tensor([[[[[ 0.9089,  2.1990, -2.5486,  ...,  0.7799,  2.6376,
             2.9988],
           [ 2.8440,  1.2185, -1.9551,  ...,  0.1864,  1.9926,
            -1.6971],
           [-0.2780,  2.0700, -2.3164,  ...,  3.4117,  1.2185,
             0.4186],
           ...,
           [ 2.4570,  1.8635,  0.8831,  ..., -1.9035,  0.9605,
            -0.7940],
           [-0.1748, -2.3422,  2.3538,  ..., -1.4391,  0.3928,
            -2.9356],
           [ 0.6767,  0.1090, -1.7487,  ...,  0.9347,  0.2896,
            -1.2585]],

          [[-0.1490, -2.1873,  1.8377,  ...,  2.6634, -1.4133,
            -0.3554],
           [-2.8324,  1.6829,  0.0058,  ...,  0.8573, -1.6713,
            -0.8198],
           [-0.0458,  1.6055,  3.2310,  ...,  3.1536,  2.2248,
             3.2826],
           ...,
           [ 2.2764, -2.8324,  2.0700,  ...,  2.8956, -2.0583,
             0.3412],
           [ 0.7799, -0.6650,  2.3538,  ...,  2.1474, -1.1037,
            -0.3554],
           [-1.3617,  1.0379,  

In [59]:
fw = get_cnn_features(net, inputs, exec_str_list)

In [60]:
if inputs.grad is not None:
    print('yes')
    inputs.grad.data.zero_()

yes


In [64]:
inputs.grad.data

tensor([[[[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           ...,
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

          [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           ...,
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

          [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           ...,
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

          ...,

          [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],

In [20]:
net.zero_grad()

In [65]:
i = 2

In [66]:
act_j = fw[i]

In [67]:
feat0_j = features[i]

In [68]:
loss_j = loss_fun(act_j, feat0_j.detach())

In [69]:
if i != 0:
    loss_j.backward(retain_graph=True)
else:
    loss_j.backward()

tensor([[[[[ 0.9089,  2.1990, -2.5486,  ...,  0.7799,  2.6376,
             2.9988],
           [ 2.8440,  1.2185, -1.9551,  ...,  0.1864,  1.9926,
            -1.6971],
           [-0.2780,  2.0700, -2.3164,  ...,  3.4117,  1.2185,
             0.4186],
           ...,
           [ 2.4570,  1.8635,  0.8831,  ..., -1.9035,  0.9605,
            -0.7940],
           [-0.1748, -2.3422,  2.3538,  ..., -1.4391,  0.3928,
            -2.9356],
           [ 0.6767,  0.1090, -1.7487,  ...,  0.9347,  0.2896,
            -1.2585]],

          [[-0.1490, -2.1873,  1.8377,  ...,  2.6634, -1.4133,
            -0.3554],
           [-2.8324,  1.6829,  0.0058,  ...,  0.8573, -1.6713,
            -0.8198],
           [-0.0458,  1.6055,  3.2310,  ...,  3.1536,  2.2248,
             3.2826],
           ...,
           [ 2.2764, -2.8324,  2.0700,  ...,  2.8956, -2.0583,
             0.3412],
           [ 0.7799, -0.6650,  2.3538,  ...,  2.1474, -1.1037,
            -0.3554],
           [-1.3617,  1.0379,  

In [58]:
inputs.grad.data

tensor([[[[[ 1.7152e-06,  2.2154e-06,  9.1221e-07,  ...,
             3.5183e-06,  4.2119e-06,  2.6723e-06],
           [ 3.2566e-06,  4.3275e-06,  1.3446e-06,  ...,
             4.4749e-06,  5.8853e-06,  3.4794e-06],
           [ 2.0538e-06,  3.2319e-06,  7.8046e-08,  ...,
             4.3650e-06,  3.8501e-06,  1.1571e-06],
           ...,
           [ 3.0824e-06,  7.0220e-06,  8.4805e-06,  ...,
             6.2518e-07,  9.1605e-07, -3.4661e-07],
           [ 3.1576e-06,  4.6877e-06,  4.0839e-06,  ...,
             2.1369e-06,  1.5047e-06,  4.4484e-08],
           [ 3.4736e-06,  4.5083e-06,  4.0610e-06,  ...,
             1.8249e-06,  1.3567e-06,  3.8152e-08]],

          [[ 1.5888e-06,  3.3746e-06,  3.1984e-06,  ...,
             1.6296e-06,  2.1492e-06,  1.5710e-06],
           [ 2.0973e-06,  5.0460e-06,  4.1286e-06,  ...,
             2.5864e-06,  3.9350e-06,  2.6733e-06],
           [ 6.0741e-07,  4.0561e-06,  3.3332e-06,  ...,
             2.8648e-06,  2.5424e-06,  1.3280e-06],
 

In [141]:
inputs.grad.data

tensor([[[[[ 1.5890e-07,  1.1575e-07, -1.0521e-08,  ...,
            -8.8557e-08, -4.0531e-08, -4.4209e-08],
           [-9.2009e-09, -1.1345e-07, -1.8678e-07,  ...,
            -1.6220e-07, -4.2980e-08, -1.5827e-08],
           [-2.7842e-07, -4.9437e-07, -5.6706e-07,  ...,
            -2.9282e-07, -1.8809e-07, -6.7007e-08],
           ...,
           [ 4.3398e-07,  2.3469e-07, -1.6532e-07,  ...,
            -4.6786e-08,  1.2877e-07,  1.2330e-07],
           [ 6.0119e-08, -8.0111e-08, -1.5942e-07,  ...,
            -1.5622e-07,  3.2350e-08, -4.5207e-08],
           [-1.7849e-08, -1.6770e-07, -1.0424e-07,  ...,
            -2.2773e-07, -4.8453e-08, -8.9762e-08]],

          [[-1.3560e-08, -1.8278e-07, -3.5389e-07,  ...,
            -7.0996e-08, -1.6435e-07, -1.3748e-07],
           [-9.0880e-08, -1.7127e-07, -8.7190e-08,  ...,
            -1.4949e-07, -2.2699e-07, -2.1197e-07],
           [-7.9127e-08, -1.9958e-07, -1.9839e-07,  ...,
            -2.5959e-07, -3.0819e-07, -2.4233e-07],
 

In [146]:
inputs.grad.data

tensor([[[[[ 1.5888e-07,  1.1576e-07, -1.0496e-08,  ...,
            -8.8618e-08, -4.0563e-08, -4.4242e-08],
           [-9.2416e-09, -1.1348e-07, -1.8676e-07,  ...,
            -1.6221e-07, -4.2973e-08, -1.5833e-08],
           [-2.7846e-07, -4.9439e-07, -5.6701e-07,  ...,
            -2.9283e-07, -1.8810e-07, -6.7028e-08],
           ...,
           [ 4.3400e-07,  2.3468e-07, -1.6535e-07,  ...,
            -4.6845e-08,  1.2874e-07,  1.2329e-07],
           [ 6.0114e-08, -8.0116e-08, -1.5939e-07,  ...,
            -1.5625e-07,  3.2351e-08, -4.5212e-08],
           [-1.7846e-08, -1.6769e-07, -1.0420e-07,  ...,
            -2.2775e-07, -4.8457e-08, -8.9767e-08]],

          [[-1.3569e-08, -1.8278e-07, -3.5388e-07,  ...,
            -7.0996e-08, -1.6435e-07, -1.3749e-07],
           [-9.0918e-08, -1.7130e-07, -8.7131e-08,  ...,
            -1.4943e-07, -2.2694e-07, -2.1198e-07],
           [-7.9142e-08, -1.9957e-07, -1.9829e-07,  ...,
            -2.5948e-07, -3.0811e-07, -2.4232e-07],
 

In [151]:
inputs.grad.data

tensor([[[[[ 8.2087e-07,  1.7298e-06,  1.6971e-06,  ...,
             1.5654e-06,  1.8048e-06,  1.0925e-06],
           [ 4.6041e-07,  1.4979e-06,  1.8540e-06,  ...,
             1.9816e-06,  1.7151e-06,  7.4789e-07],
           [-2.9448e-07,  2.2612e-07,  4.0753e-07,  ...,
             1.3781e-06,  9.1454e-07,  1.3414e-07],
           ...,
           [ 5.2293e-07,  1.5344e-06,  1.2499e-06,  ...,
             4.4707e-07,  6.6324e-07,  2.8993e-07],
           [ 2.3360e-07,  1.1845e-06,  1.0167e-06,  ...,
             4.1183e-07,  6.7361e-07,  1.2840e-07],
           [ 1.2612e-07,  7.9600e-07,  2.9255e-07,  ...,
            -3.7516e-08,  2.0558e-07, -3.5681e-07]],

          [[ 4.4745e-07,  7.2795e-07,  3.3636e-07,  ...,
             1.3490e-06,  1.5020e-06,  7.6782e-07],
           [ 6.9125e-07,  1.5134e-06,  1.7443e-06,  ...,
             2.2415e-06,  1.6335e-06,  3.0930e-07],
           [ 8.2041e-07,  1.5497e-06,  1.2788e-06,  ...,
             2.1409e-06,  1.4944e-06,  2.6707e-07],
 

In [42]:
net.module.bn1.

BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

'0.4.0'